1. Split MIBiG compounds into 5 groups for cross-validation based on BigScape families.

In [4]:
from math import ceil
from pathlib import Path
import pandas as pd

nerpa_dir = Path('/home/ilianolhin/git/nerpa2')

mibig_bgcs_table = pd.read_csv(nerpa_dir / 'scripts/build_mibig_info_table/mibig_bgcs_info.tsv', sep='\t')
mibig_compounds_approved = mibig_bgcs_table[mibig_bgcs_table["in_approved_matches"] == True].copy()

if mibig_compounds_approved["bigscape_families"].isna().sum() > 0:
    print('Warning! Missing bigscape_families values present.')

# Aggregate BGCs by BigScape family using groupby
family_to_bgcs = mibig_compounds_approved.groupby("bigscape_families")["bgc_id"].apply(list).to_dict()

n_groups = 5  # Number of groups to split the families into
total_bgcs = sum(len(bgcs) for bgcs in family_to_bgcs.values())
target_group_size = ceil(total_bgcs / n_groups)

family_iter = iter(family_to_bgcs.items())
family_to_sample_group = {}

for group_idx in range(n_groups):
    group_size = 0
    while group_size < target_group_size and (family := next(family_iter, None)) is not None:
        family_name, bgcs = family
        family_to_sample_group[family_name] = group_idx
        group_size += len(bgcs)

# q: assert that all families are assigned to a group
assert len(family_to_sample_group) == len(family_to_bgcs), "Not all families are assigned to a group"

# Assign sample group to each row
mibig_compounds_approved["sample_group"] = (mibig_compounds_approved["bigscape_families"]
                                            .map(family_to_sample_group)
                                            .astype('Int64'))

# display mibig_compounds_approved
print(mibig_compounds_approved.head())

        bgc_id  num_a_domains origins   bigscape_families   compound_id  \
15  BGC0000296              6  mibig3  Others_274_mix_274  BGC0000296.0   
27  BGC0000305             11  mibig3  Others_282_mix_282  BGC0000305.0   
28  BGC0000306              6  mibig3  Others_283_mix_283  BGC0000306.0   
29  BGC0000307              9  mibig3  Others_284_mix_284  BGC0000307.0   
34  BGC0000310             12  mibig3  Others_287_mix_287  BGC0000310.0   

    num_recognized_nodes  iso_class_idx  in_approved_matches  \
15                    11           3006                 True   
27                    11           1782                 True   
28                     6            368                 True   
29                     9           3138                 True   
34                    12            612                 True   

    deprecated in mibig4  sample_group  
15                 False             3  
27                 False             3  
28                 False             3  


2. Form subsets of approved matches with MIBiG compounds for cross-validation.

In [5]:
from itertools import chain
from src.matching.match_type import Match
approved_matches_txt = Path(nerpa_dir / 'test_data/approved_matches/approved_matches.txt')
approved_matches_strs = approved_matches_txt.read_text().split('\n\n')
approved_matches = (Match.from_str(match_str)
                    for match_str in approved_matches_strs
                    if match_str.strip())
approved_matches_per_compound = {match.nrp_variant_id.nrp_id: match
                                 for match in approved_matches}

benchmarking_dir = Path(nerpa_dir / 'benchmarking')
(benchmarking_dir / 'approved_matches_subsets').mkdir(parents=True, exist_ok=True)

training_subset_paths = [benchmarking_dir / 'approved_matches_subsets' / f'training_subset_{i}.txt'
                         for i in range(n_groups)]
testing_subset_paths = [benchmarking_dir / 'approved_matches_subsets' / f'testing_subset_{i}.txt'
                        for i in range(n_groups)]

subsets = mibig_compounds_approved.groupby('sample_group')['compound_id'].apply(list).to_dict()

if set(approved_matches_per_compound.keys()) != set(chain(*subsets.values())):
    print('Warning!\n'
          "The set of approved matches does not match the set of MIBiG compounds in subsets.\n"
          f"subsets \ approved_matches = {set(chain(*subsets.values())) - set(approved_matches_per_compound.keys())}\n"
          f"approved_matches \ subsets = {set(approved_matches_per_compound.keys()) - set(chain(*subsets.values()))}")

for group_idx, compound_ids in subsets.items():
    testing_subset = [approved_matches_per_compound[nrp_id]
                      for nrp_id in compound_ids]
    training_subset = [match
                       for match in approved_matches_per_compound.values()
                       if match.nrp_variant_id.nrp_id not in compound_ids]

    # Save the subset to a file
    with open(testing_subset_paths[group_idx], 'w') as f:
        f.write('\n'.join(match.nrp_variant_id.nrp_id
                          for match in testing_subset))

    with open(training_subset_paths[group_idx], 'w') as f:
        f.write('\n\n'.join(map(str, training_subset)))

Warning!
The set of approved matches does not match the set of MIBiG compounds in subsets.
subsets \ approved_matches = set()
approved_matches \ subsets = {'BGC0000405.3', 'BGC0002061.0', 'BGC0002061.1', 'BGC0000314.0', 'BGC0000368.0', 'BGC0000289.0'}


3. Create a config file for each subset.

In [6]:
import subprocess
(benchmarking_dir / 'training_results').mkdir(parents=True, exist_ok=True)
training_results_paths = [benchmarking_dir / 'training_results' / f'subset_{i}'
                          for i in range(n_groups)]
for subset_idx in range(n_groups):
    command = (f'python {nerpa_dir / "train_nerpa.py"} '
               f'--approved-matches {training_subset_paths[subset_idx]} '
               f'--output-dir {training_results_paths[subset_idx]}')
    print(f'Running command: {command}')
    subprocess.run(command, shell=True, check=True)

Running command: python /home/ilianolhin/git/nerpa2/train_nerpa.py --approved-matches /home/ilianolhin/git/nerpa2/benchmarking/approved_matches_subsets/training_subset_0.txt --output-dir /home/ilianolhin/git/nerpa2/benchmarking/training_results/subset_0
Loading approved matches
Loading BGC variants
Inferring edge parameters from counts...
WARNING(ok): not enough data for INSERT_AT_START, so I use the same as for INSERT
WARNING(ok): not enough data for influence of PKS_DOWNSTREAM on START_INSERTING, so I ignore it
WARNING(ok): not enough data for influence of PKS_DOWNSTREAM on START_INSERTING_AT_END, so I ignore it
Building step function...
Calculating modifications frequencies...
Calculating PKS probability...
Unknown residue predictions correctness: [(0.008, False), (0.008, False), (0.008, False), (0.008, False), (0.008, False), (0.008, False), (0.008, False), (0.008, False), (0.008, True), (0.008, False), (0.008, True), (0.008, True), (0.008, False), (0.008, False), (0.008, False), (

4. Prepare inputs for Nerpa

In [7]:
import shutil

inputs_dir = benchmarking_dir / 'cross_validation_inputs'
inputs_dir.mkdir(parents=True, exist_ok=True)
as_results_on_mibig_dir = Path('/home/ilianolhin/programs/antismash7/as_results_mibig4_nrps')
compounds_table = pd.read_csv(nerpa_dir / 'data/pnrpdb2rc1_summary.tsv', sep='\t')

for subset_idx in range(n_groups):
    subset_input_dir = inputs_dir / f'subset_{subset_idx}'
    # 1. copy configs from training results
    source = training_results_paths[subset_idx] / 'new_configs'
    dest = subset_input_dir / 'configs'
    dest.mkdir(parents=True, exist_ok=True)
    shutil.copytree(source, dest, dirs_exist_ok=True)

    # 2. Load nrp_ids for testing
    nrp_ids = [nrp_id.strip()
               for nrp_id in testing_subset_paths[subset_idx].read_text().split('\n')
               if nrp_id.strip()]
    bgc_ids = {nrp_id.split('.')[0] for nrp_id in nrp_ids}

    # 2. Create a file with antismash paths
    antismash_paths = [as_results_on_mibig_dir / bgc_id
                for bgc_id in bgc_ids]
    with open(subset_input_dir / 'antismash_paths.txt', 'w') as f:
        f.write('\n'.join(str(path) for path in antismash_paths))

    # 3. Create a file with MIBiG compounds
    # q: assert that all nrp_ids are in compounds_table
    assert set(nrp_ids).issubset(set(compounds_table['ID'])), \
        "Not all nrp_ids are in compounds_table"

    mibig_compounds = compounds_table[compounds_table['ID'].isin(nrp_ids)]
    mibig_compounds.to_csv(subset_input_dir / 'compounds.tsv',
                           sep='\t', index=False)



In [8]:
def is_from_mibig_norine(nrp_id: str) -> bool:
    return nrp_id.startswith('BGC') or nrp_id.startswith('NOR')

def at_least_3_recognized_monomers(nrp_id: str) -> bool:
    """Check if the nrp_id has at least 3 recognized monomers in the compound_stats_table."""
    values = compound_stats_table[compound_stats_table['compound_id'] == nrp_id]['num_recognized_nodes'].values
    return values.size > 0 and values[0] >= 3

compound_stats_table = pd.read_csv(nerpa_dir / 'scripts/build_mibig_info_table/rban_graphs_filtered.tsv', sep='\t')
mibig_norine_compounds = (compounds_table[compounds_table['ID']
                        .apply(lambda nrp_id: is_from_mibig_norine(nrp_id) and at_least_3_recognized_monomers(nrp_id))])

mibig_norine_compounds.to_csv(nerpa_dir / 'data' / 'mibig_norine_compounds.tsv',
                               sep='\t', index=False)

5. Run Nerpa on each subset.

In [4]:
import subprocess
from pathlib import Path
nerpa_dir = Path('/home/ilianolhin/git/nerpa2')
benchmarking_dir = Path(nerpa_dir / 'benchmarking')
inputs_dir = benchmarking_dir / 'cross_validation_inputs'
n_groups = 5
results_dir = benchmarking_dir / 'cross_validation_results'
results_dir.mkdir(parents=True, exist_ok=True)

for subset_idx in range(n_groups):
    subset_input_dir = inputs_dir / f'subset_{subset_idx}'
    command = (f'python {nerpa_dir / "nerpa.py"} '
               f'--antismash-paths-file {subset_input_dir / "antismash_paths.txt"} '
               f'--rban-json {nerpa_dir / "data/mibig_norine_rban_preprocessed.json"} '
               f'--configs-dir {subset_input_dir / "configs"} '
               '--process-hybrids '
               '--max-num-matches-per-bgc 10 '
               '--max-num-matches 0 '
               f'--output-dir {results_dir / f"subset_{subset_idx}"} '
               f'--force-output-dir '
               '--fast-matching '
               '--threads 5 '
               '--skip-molecule-drawing')
    print(f'Running command: {command}')
    subprocess.run(command, shell=True, check=True)

Running command: python /home/ilianolhin/git/nerpa2/nerpa.py --antismash-paths-file /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_inputs/subset_0/antismash_paths.txt --rban-json /home/ilianolhin/git/nerpa2/data/mibig_norine_rban_preprocessed.json --configs-dir /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_inputs/subset_0/configs --process-hybrids --max-num-matches-per-bgc 10 --max-num-matches 0 --output-dir /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_0 --force-output-dir --fast-matching --threads 5 --skip-molecule-drawing

Started with command: /home/ilianolhin/git/nerpa2/nerpa.py --antismash-paths-file /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_inputs/subset_0/antismash_paths.txt --rban-json /home/ilianolhin/git/nerpa2/data/mibig_norine_rban_preprocessed.json --configs-dir /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_inputs/subset_0/configs --process-hybrids --max-num-matches-per-bgc 10 --max-num-matches 

[20:10:22] Can't kekulize mol.  Unkekulized atoms: 6 7 8 11 12
[20:10:22] non-ring atom 1 marked aromatic
[20:10:22] non-ring atom 0 marked aromatic
[20:10:22] Explicit valence for atom # 7 N, 4, is greater than permitted
[20:10:22] Can't kekulize mol.  Unkekulized atoms: 2 3 4
[20:10:22] Explicit valence for atom # 0 C, 5, is greater than permitted
[20:10:22] non-ring atom 6 marked aromatic
[20:10:22] non-ring atom 6 marked aromatic
[20:10:22] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 10 marked aromatic
[20:10:22] non-ring atom 4 marked aromatic
[20:10:22] non-ring atom 4 marked aromatic
[20:10:22] non-ring atom 0 marked aromatic
[20:10:22] Explicit valence for atom # 0 C, 5, is greater than


=== Resolving NRP-PK hybrid monomers candidates

== Running: java -jar /home/ilianolhin/git/nerpa2/external_tools/rBAN/rBAN-1.0.jar -inputFile /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_0/hybrid_monomers_input.json -outputFolder /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_0/ -outputFileName hybrid_monomers_output.json -monomersDB /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_0/rban_monomers_db.json

  WARNING WRONG SMILES: 539_16 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 540_6 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 819_3 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  Process completed successfully. Compounds analysed: 294

== Done


======= Done with Structures preprocessing with rBAN

======= Processing rBAN output
results wi

[20:11:03] Can't kekulize mol.  Unkekulized atoms: 6 7 8 11 12
[20:11:03] non-ring atom 1 marked aromatic
[20:11:03] non-ring atom 0 marked aromatic
[20:11:03] Explicit valence for atom # 7 N, 4, is greater than permitted
[20:11:03] Can't kekulize mol.  Unkekulized atoms: 2 3 4
[20:11:03] Explicit valence for atom # 0 C, 5, is greater than permitted
[20:11:03] non-ring atom 6 marked aromatic
[20:11:03] non-ring atom 6 marked aromatic
[20:11:03] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 10 marked aromatic
[20:11:03] non-ring atom 4 marked aromatic
[20:11:03] non-ring atom 4 marked aromatic
[20:11:03] non-ring atom 0 marked aromatic
[20:11:03] Explicit valence for atom # 0 C, 5, is greater than


=== Resolving NRP-PK hybrid monomers candidates

== Running: java -jar /home/ilianolhin/git/nerpa2/external_tools/rBAN/rBAN-1.0.jar -inputFile /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_1/hybrid_monomers_input.json -outputFolder /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_1/ -outputFileName hybrid_monomers_output.json -monomersDB /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_1/rban_monomers_db.json

  WARNING WRONG SMILES: 539_16 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 540_6 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 819_3 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  Process completed successfully. Compounds analysed: 294

== Done


======= Done with Structures preprocessing with rBAN

======= Processing rBAN output
results wi

...


=== Resolving NRP-PK hybrid monomers candidates

== Running: java -jar /home/ilianolhin/git/nerpa2/external_tools/rBAN/rBAN-1.0.jar -inputFile /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_2/hybrid_monomers_input.json -outputFolder /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_2/ -outputFileName hybrid_monomers_output.json -monomersDB /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_2/rban_monomers_db.json

  WARNING WRONG SMILES: 539_16 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 540_6 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 819_3 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  Process completed successfully. Compounds analysed: 294

== Done


======= Done with Structures preprocessing with rBAN

======= Processing rBAN output
results wi

[20:12:24] Can't kekulize mol.  Unkekulized atoms: 6 7 8 11 12
[20:12:24] non-ring atom 1 marked aromatic
[20:12:24] non-ring atom 0 marked aromatic
[20:12:24] Explicit valence for atom # 7 N, 4, is greater than permitted
[20:12:24] Can't kekulize mol.  Unkekulized atoms: 2 3 4
[20:12:24] Explicit valence for atom # 0 C, 5, is greater than permitted
[20:12:24] non-ring atom 6 marked aromatic
[20:12:24] non-ring atom 6 marked aromatic
[20:12:24] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 10 marked aromatic
[20:12:24] non-ring atom 4 marked aromatic
[20:12:24] non-ring atom 4 marked aromatic
[20:12:24] non-ring atom 0 marked aromatic
[20:12:24] Explicit valence for atom # 0 C, 5, is greater than


=== Resolving NRP-PK hybrid monomers candidates

== Running: java -jar /home/ilianolhin/git/nerpa2/external_tools/rBAN/rBAN-1.0.jar -inputFile /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_3/hybrid_monomers_input.json -outputFolder /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_3/ -outputFileName hybrid_monomers_output.json -monomersDB /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_3/rban_monomers_db.json

  WARNING WRONG SMILES: 539_16 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 540_6 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 819_3 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  Process completed successfully. Compounds analysed: 294

== Done


======= Done with Structures preprocessing with rBAN

======= Processing rBAN output
results wi

[20:13:04] Can't kekulize mol.  Unkekulized atoms: 6 7 8 11 12
[20:13:04] non-ring atom 1 marked aromatic
[20:13:04] non-ring atom 0 marked aromatic
[20:13:04] Explicit valence for atom # 7 N, 4, is greater than permitted
[20:13:04] Can't kekulize mol.  Unkekulized atoms: 2 3 4
[20:13:04] Explicit valence for atom # 0 C, 5, is greater than permitted
[20:13:04] non-ring atom 6 marked aromatic
[20:13:04] non-ring atom 6 marked aromatic
[20:13:04] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 10 marked aromatic
[20:13:04] non-ring atom 4 marked aromatic
[20:13:04] non-ring atom 4 marked aromatic
[20:13:04] non-ring atom 0 marked aromatic
[20:13:04] Explicit valence for atom # 0 C, 5, is greater than


=== Resolving NRP-PK hybrid monomers candidates

== Running: java -jar /home/ilianolhin/git/nerpa2/external_tools/rBAN/rBAN-1.0.jar -inputFile /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_4/hybrid_monomers_input.json -outputFolder /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_4/ -outputFileName hybrid_monomers_output.json -monomersDB /home/ilianolhin/git/nerpa2/benchmarking/cross_validation_results/subset_4/rban_monomers_db.json

  WARNING WRONG SMILES: 539_16 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 540_6 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  WARNING WRONG SMILES: 819_3 could not parse 'Nc1cc(Cl)ccc1(=O)O', a valid kekulé structure could not be assigned
  Process completed successfully. Compounds analysed: 294

== Done


======= Done with Structures preprocessing with rBAN

======= Processing rBAN output
results wi